<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/master/EEC_elmo_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
t = np.transpose
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
##Install flair

!pip install -q flair allennlp

    100% |████████████████████████████████| 81kB 3.1MB/s 
    100% |████████████████████████████████| 5.6MB 6.0MB/s 
    100% |████████████████████████████████| 798kB 23.1MB/s 
    100% |████████████████████████████████| 122kB 30.3MB/s 
    100% |████████████████████████████████| 512kB 6.7MB/s 
    100% |████████████████████████████████| 4.5MB 6.6MB/s 
    100% |████████████████████████████████| 133kB 32.3MB/s 
    100% |████████████████████████████████| 51kB 20.4MB/s 
    100% |████████████████████████████████| 1.5MB 16.7MB/s 
    100% |████████████████████████████████| 51kB 21.9MB/s 
    100% |████████████████████████████████| 552kB 25.6MB/s 
    100% |████████████████████████████████| 1.0MB 16.6MB/s 
    100% |████████████████████████████████| 51kB 19.3MB/s 
    100% |████████████████████████████████| 2.3MB 11.8MB/s 
    100% |████████████████████████████████| 61kB 24.1MB/s 
    100% |████████████████████████████████| 143kB 32.5MB/s 
    100% |████████████████████████████████| 51kB 

In [2]:
##Using flair to load emlo small embeddinngs, which is in 768 dimensions
from flair.embeddings import BertEmbeddings, ELMoEmbeddings
from flair.data import Sentence


# init embedding
embedding = ELMoEmbeddings('small') #BertEmbeddings('bert-large-uncased')


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
##Load EEC Data
!wget http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip
!unzip Equity-Evaluation-Corpus.zip
##Load Packages
import numpy as np
import scipy, requests, codecs, os, re, nltk, itertools, csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering, KMeans
import tensorflow as tf
from scipy.stats import spearmanr
import pandas as pd
import functools as ft
import os
import io
nltk.download('punkt')

##Read EEC
EEC = pd.read_csv('/content/Equity-Evaluation-Corpus.csv', header=0)


--2019-03-02 13:37:42--  http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip
Resolving www.saifmohammad.com (www.saifmohammad.com)... 192.185.17.122
Connecting to www.saifmohammad.com (www.saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102568 (100K) [application/zip]
Saving to: ‘Equity-Evaluation-Corpus.zip.1’

Equity-Evaluation-C 100%[===================>] 100.16K  --.-KB/s    in 0.1s    

2019-03-02 13:37:42 (914 KB/s) - ‘Equity-Evaluation-Corpus.zip.1’ saved [102568/102568]

Archive:  Equity-Evaluation-Corpus.zip
replace Equity-Evaluation-Corpus.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Racial Debiasing

In [0]:
##Load male names in the EEC paper
word_list_MALE = ['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance',
       'Darnell', 'Lamar', 'Malik', 'Terrence', 'Adam', 'Harry', 'Josh',
       'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack']

In [34]:
## Get racial names from web
!wget https://gist.githubusercontent.com/mbejda/61eb488cec271086632d/raw/6340b8045b28c2abc0b1d44cfbc80f40284ef890/Black-Male-Names.csv
!wget https://gist.githubusercontent.com/mbejda/6c2293ba3333b7e76269/raw/60aa0c95e8ee9b11b915a26f47480fef5c3203ed/White-Male-Names.csv
!wget https://gist.githubusercontent.com/mbejda/26ad0574eda7fca78573/raw/6936d1a8f5fa5220f2f60a51a06a35b172c50f93/White-Female-Names.csv
!wget https://gist.githubusercontent.com/mbejda/9dc89056005a689a6456/raw/bb6ef2375f1289d0ef10dbd8e9469670ac23ceab/Black-Female-Names.csv


--2019-03-02 14:12:49--  https://gist.githubusercontent.com/mbejda/61eb488cec271086632d/raw/6340b8045b28c2abc0b1d44cfbc80f40284ef890/Black-Male-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 921600 (900K) [text/plain]
Saving to: ‘Black-Male-Names.csv’

Black-Male-Names.cs 100%[===================>] 900.00K  --.-KB/s    in 0.07s   

2019-03-02 14:12:49 (12.1 MB/s) - ‘Black-Male-Names.csv’ saved [921600/921600]

--2019-03-02 14:12:50--  https://gist.githubusercontent.com/mbejda/6c2293ba3333b7e76269/raw/60aa0c95e8ee9b11b915a26f47480fef5c3203ed/White-Male-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)

# Load racial names from web

In [0]:
## Load names from web, don't run it if using EEC names
word_list_male =[]

In [0]:
f = open('/content/White-Male-Names.csv', 'r+')
f.readline()
i=0
for line in f:
  i +=1
  word_list_male.append(line.rstrip().split(',')[1].split(' ')[1])
  if i>4000:
    break
f.close()

In [0]:
f = open('/content/Black-Male-Names.csv', 'r+')
i=0
f.readline()
for line in f:
  i+=1
  word_list_male.append(line.rstrip().split(',')[1].split(' ')[1])
  if i>4000:
    break
f.close()

In [0]:
word_list_female = []
f = open('/content/White-Female-Names.csv', 'r+')
f.readline()
for line in f:
  word_list_female.append(line.rstrip().split(',')[1].split(' ')[1])
f.close()

In [0]:
f = open('/content/Black-Female-Names.csv', 'r+')
f.readline()
for line in f:
  word_list_female.append(line.rstrip().split(',')[1].split(' ')[1])
f.close()

# Get CN embeddings

In [0]:
from sklearn.decomposition import TruncatedSVD, randomized_svd
from numpy.linalg import norm

In [6]:
## Get brown_emb 
!gdown https://drive.google.com/uc?id=17TK2h3cz7amgm2mCY4QCYy1yh23ZFWDU
!pip3 install -q pymagnitude
from pymagnitude import *
from pymagnitude import MagnitudeUtils
import numpy as np
import pickle
with open('/content/elmo_embeddings_emma_brown.pkl', 'rb') as f:
    elmo_data = pickle.load(f)
elmo_data = elmo_data['brown_embs']

## Get brown corpus
import nltk
nltk.download('brown')
from nltk.corpus import brown
brown_corpus = brown.sents()

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [0]:
def pick_embeddings(corpus, sent_embs, word_list):
    X = []
    #labels = []
    #sents = []
    for i, s in enumerate(corpus):
       # print(s)
        for j, w in enumerate(s):
            if w in word_list:
                X.append(sent_embs[i][j])
                #labels.append(w)
                #sents.append(s)
    return X


In [0]:
## Get elmo word embeddings of words(male names) to find conceptors
X, labels, _ = pick_embeddings(brown_corpus, elmo_data, word_list_male)
cn_embeddings = np.array(X)

In [0]:
## Since conceptors doesn't change in the loop, I calculated conceptor negation matrix beforehand to reduce runtime.
alpha = 1
R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]   
C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(768))# calculate the conceptor matrix
negC = np.eye(768) - C

# Run experiment

In [0]:
def run_raw(sen_emb, sent1, sent2):
  
  sentence1 = Sentence(sent1)
  sentence2 = Sentence(sent2)
  x = embedding.embed(sentence1) # get sentence embedding of sentence 1
  se = torch.stack([token.embedding for token in sentence1])
  sent1_emb = se.numpy()
  x = embedding.embed(sentence2) # get sentence embedding of sentence 2
  se = torch.stack([token.embedding for token in sentence2])
  sent2_emb = se.numpy()
  sent1_emb = np.average(sent1_emb, axis = 0) #average sentence1 embeddings
  sent2_emb = np.average(sent2_emb, axis = 0) #average sentence2 embeddings
  proj1 = (sent1_emb/norm(sent1_emb)).dot(sen_emb)/norm(sen_emb) #cosine similarity
  proj2 = (sent2_emb/norm(sent2_emb)).dot(sen_emb)/norm(sen_emb)
  return proj1, proj2

In [0]:
def run_conceptor(sen_emb, sent1, sent2):
  sentence1 = Sentence(sent1) 
  sentence2 = Sentence(sent2)
  x = embedding.embed(sentence1) # get sentence embedding of sentence 1
  se = torch.stack([token.embedding for token in sentence1])
  sent1_emb = se.numpy()
  x = embedding.embed(sentence2) # get sentence embedding of sentence 2
  se = torch.stack([token.embedding for token in sentence2])
  sent2_emb = se.numpy()
  sent1_emb = np.average(sent1_emb, axis = 0) #average sentence1 embeddings
  sent2_emb = np.average(sent2_emb, axis = 0) #average sentence2 embeddings
  sent1_emb = negC.dot(sent1_emb.T).T  # CN sentence embeddings
  sent2_emb = negC.dot(sent2_emb.T).T
  sen_emb =  negC.dot(sen_emb.T).T
  proj1 = (sent1_emb/norm(sent1_emb)).dot(sen_emb)/norm(sen_emb) #cosine similarity
  proj2 = (sent2_emb/norm(sent2_emb)).dot(sen_emb)/norm(sen_emb)
  return proj1, proj2

In [45]:
# Get male sentences results
p= []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen] # get sentence of certain sentiment
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='male')] #get AA male sentences
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='male') ] #get EA male sentences
  sen_emb  = pick_embeddings(brown_corpus, elmo_data, str(sen)) # get certain sentiment embedding
  sen_emb = np.average(sen_emb, axis = 0)
  print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    
    score1, score2 = run_raw(sen_emb, sen1, sen2) #projection score on raw data
    score3, score4 = run_conceptor(sen_emb, sen1, sen2) # projection score on CN
    aa.append(score1) # raw score for african american
    ea.append(score2) # raw score for european american
    aa_cn.append(score3) # CN score for african american
    ea_cn.append(score4) # CN score for european american
  #paired t-test for raw data
  p1 = scipy.stats.ttest_rel(aa, ea)
  #paired t-test for CN data
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


350
350
350
350


In [50]:
print('RAW vs CN data using ELMO, racial debiasing, male')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)


RAW vs CN data using ELMO, male 4k names
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger          -6.900955  2.442959e-11       -2.499675  1.288927e-02
fear           -4.780334  2.583952e-06        0.745871  4.562475e-01
joy            -9.062251  9.333792e-18      -11.695337  6.942281e-27
sadness        -4.793218  2.432981e-06       -1.676787  9.447953e-02


In [0]:
# Load female names in EEC paper, don't run it if using web names
word_list_female= ['Nichelle',
       'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha',
       'Tia', 'Lakisha', 'Latoya', 'Amanda', 'Courtney', 'Heather',
       'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie',
       'Ellen']

In [0]:
## Get elmo word embeddings of words(female names) to find conceptors
X = pick_embeddings(brown_corpus, elmo_data,word_list_female)
cn_embeddings = np.array(X)

In [0]:
alpha = 1
R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]   
C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(768))# calculate the conceptor matrix
negC = np.eye(768) - C

In [63]:
## Get female sentences result
p= []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen] # get sentence of certain sentiment
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')] #get AA female sentences
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ] #get EA female sentences
  sen_emb  = pick_embeddings(brown_corpus, elmo_data, str(sen)) # get certain sentiment embedding
  sen_emb = np.average(sen_emb, axis = 0)
  print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    
    score1, score2 = run_raw(sen_emb, sen1, sen2) #projection score on raw data
    score3, score4 = run_conceptor(sen_emb, sen1, sen2) # projection score on CN
    aa.append(score1) # raw score for african american
    ea.append(score2) # raw score for european american
    aa_cn.append(score3) # CN score for african american
    ea_cn.append(score4) # CN score for european american
  #paired t-test for raw data
  p1 = scipy.stats.ttest_rel(aa, ea)
  #paired t-test for CN data
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


350
350
350
350


In [64]:
len(word_list_female)

7038

In [65]:
print('RAW vs CN data using ELMO, female names from EEC, alpha = 1')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)


RAW vs CN data using ELMO, female 4k names, alpha = 1
         raw_t_statistic  raw_p_value  cn_t_statistic  cn_p_value
anger          -1.533890     0.125963       -4.638600    0.000005
fear           -1.382389     0.167736       -3.946651    0.000096
joy             0.668063     0.504535       -0.476559    0.633974
sadness        -1.684075     0.093061       -4.225553    0.000030


# Gender Debiasing

# Load gender names

# Using names from web

In [66]:
## Get large gender name lists from web, don't run it if using names only in EEC paper
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt

--2019-03-02 14:41:11--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20466 (20K) [text/plain]
Saving to: ‘male.txt’

male.txt            100%[===================>]  19.99K  --.-KB/s    in 0.04s   

2019-03-02 14:41:12 (556 KB/s) - ‘male.txt’ saved [20466/20466]

--2019-03-02 14:41:13--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35751 (35K) [text/plain]
Saving to: ‘female.txt’

female.txt          100%[===================>]  34.91K  --.-KB/s    in 0.04s   

2019-03-02 14:41:13 (964 KB/s) - ‘female.txt’ saved [35751/35751]



In [0]:
f = open('/content/male.txt', 'r+')
i = 0
word_list_gender= []
for line in f:
  i+=1
  if i < 7:
    continue
  token = line.split()
  word_list_gender.append(token[0])
f.close()
#load names from larger name dataset
f = open('/content/female.txt', 'r+')
i = 0
for line in f:
  i+=1
  if i < 7:
    continue
  token = line.split()
  word_list_gender.append(token[0])
f.close()

In [76]:
len(word_list_gender)

7944

In [0]:
X = pick_embeddings(brown_corpus, elmo_data,word_list_gender)
cn_embeddings = np.array(X)
alpha = 1
R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]   
C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(768))# calculate the conceptor matrix
negC = np.eye(768) - C

In [0]:
## Get female sentences result
p= []
for sen in ['anger','fear','joy', 'sadness']:
  ma = []
  fe = []
  ma_cn = []
  fe_cn = []
  sent_sent = EEC[EEC['Emotion']==sen] # get sentence of certain sentiment
  sent_ma = sent_sent[(sent_sent['Gender']=='male')] #get male sentences
  sent_fe = sent_sent[(sent_sent['Gender']=='female') ] #get female sentences
  sen_emb  = pick_embeddings(brown_corpus, elmo_data, str(sen)) # get certain sentiment embedding
  sen_emb = np.average(sen_emb, axis = 0)
  for sen1, sen2 in zip(sent_ma['Sentence'], sent_fe['Sentence']):
    
    score1, score2 = run_raw(sen_emb, sen1, sen2) #projection score on raw data
    score3, score4 = run_conceptor(sen_emb, sen1, sen2) # projection score on CN
    ma.append(score1) # raw score for male 
    fe.append(score2) # raw score for female
    ma_cn.append(score3) # CN score for male
    fe_cn.append(score4) # CN score for female
  #paired t-test for raw data
  p1 = scipy.stats.ttest_rel(ma, fe)
  #paired t-test for CN data
  p2 = scipy.stats.ttest_rel(ma_cn, fe_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [75]:
print('Gender Debiasing, RAW vs CN data using ELMO, using 8k gender names, alpha = 1')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)


Gender Debiasing, RAW vs CN data using ELMO, alpha = 1
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger           2.546938  1.100916e-02       -8.666059  1.670581e-17
fear           -0.861152  3.893512e-01       -8.668266  1.640657e-17
joy            -7.560719  8.726220e-14        9.273436  9.974654e-20
sadness        -0.986186  3.242694e-01       -8.847929  3.716502e-18


In [77]:
EEC['Person'].unique()

array(['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance',
       'Darnell', 'Lamar', 'Malik', 'Terrence', 'Adam', 'Harry', 'Josh',
       'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack', 'he',
       'this man', 'this boy', 'my brother', 'my son', 'my husband',
       'my boyfriend', 'my father', 'my uncle', 'my dad', 'Nichelle',
       'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha',
       'Tia', 'Lakisha', 'Latoya', 'Amanda', 'Courtney', 'Heather',
       'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie',
       'Ellen', 'she', 'this woman', 'this girl', 'my sister',
       'my daughter', 'my wife', 'my girlfriend', 'my mother', 'my aunt',
       'my mom', 'him', 'her'], dtype=object)

# Using names in paper

In [0]:
# Names in EEC paper
word_list_gender = EEC['Person'].unique()

In [84]:
len(word_list_gender)

62

In [0]:
X = pick_embeddings(brown_corpus, elmo_data,word_list_gender)
cn_embeddings = np.array(X)
alpha = 1
R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]   
C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(768))# calculate the conceptor matrix
negC = np.eye(768) - C

In [0]:
## Get female sentences result
p= []
for sen in ['anger','fear','joy', 'sadness']:
  ma = []
  fe = []
  ma_cn = []
  fe_cn = []
  sent_sent = EEC[EEC['Emotion']==sen] # get sentence of certain sentiment
  sent_ma = sent_sent[(sent_sent['Gender']=='male')] #get male sentences
  sent_fe = sent_sent[(sent_sent['Gender']=='female') ] #get female sentences
  sen_emb  = pick_embeddings(brown_corpus, elmo_data, str(sen)) # get certain sentiment embedding
  sen_emb = np.average(sen_emb, axis = 0)
  for sen1, sen2 in zip(sent_ma['Sentence'], sent_fe['Sentence']):
    
    score1, score2 = run_raw(sen_emb, sen1, sen2) #projection score on raw data
    score3, score4 = run_conceptor(sen_emb, sen1, sen2) # projection score on CN
    ma.append(score1) # raw score for male
    fe.append(score2) # raw score for female
    ma_cn.append(score3) # CN score for male
    fe_cn.append(score4) # CN score for female
  #paired t-test for raw data
  p1 = scipy.stats.ttest_rel(ma, fe)
  #paired t-test for CN data
  p2 = scipy.stats.ttest_rel(ma_cn, fe_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [85]:
print('Gender Debiasing, RAW vs CN data using ELMO, using 62 gender names and pronouns in EEC, alpha = 1')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

Gender Debiasing, RAW vs CN data using ELMO, using 62 gender names and pronouns in EEC, alpha = 1
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger           2.546525  1.102211e-02       -6.406934  2.238115e-10
fear           -0.861154  3.893501e-01       -8.237407  5.199210e-16
joy            -7.560718  8.726258e-14       -6.488792  1.331473e-10
sadness        -0.986187  3.242686e-01       -8.491553  6.893384e-17
